# Exercise 3

Exercise 3 includes four problems that you need to solve with programming, and by providing answers to questions. For each problem you need to modify the notebook by adding your own solutions. Remember to save and commit your changes locally, and push your changes to GitHub after each major change! Regular commits will help you to keep track of your changes (and revert them if needed). Pushing your work to GitHub will ensure that you don't lose any work in case your computer crashes (can happen!).

### Time allocation

**Completing this exercise takes approximately: 6-10 hours** (based on previous year statistics). However, the time it takes can vary significantly from student to student, so we recommended that you start immediately working on the exercise to avoid issues meeting with the deadline.  

### Due date

You should submit (push) your Exercise answers to your personal Github repository approx. **2.5weeks after the practical session (submit by Sunday  26th Nov, 23:59)**. 
      
### Start your exercise in CSC Notebooks

Before you can start programming, you need to launch the CSC Notebook instance and clone your **personal copy of the Exercise repository** (i.e. something like `exercise-3-htenkanen`) there using Git. If you need help with this, [read the documentation on the course site](https://spatial-analytics.readthedocs.io/en/latest/lessons/L1/git-basics.html).

### Working with Jupyter Notebooks

Jupyter Notebooks are documents that can be used and run inside the JupyterLab programming environment (e.g. at [notebooks.csc.fi](https://notebooks.csc.fi/)) containing the computer code and rich text elements (such as text, figures, tables and links). 

**A couple of hints**:

- You can **execute a cell** by clicking a given cell that you want to run and pressing <kbd>Shift</kbd> + <kbd>Enter</kbd> (or by clicking the "Play" button on top)
- You can **change the cell-type** between `Markdown` (for writing text) and `Code` (for writing/executing code) from the dropdown menu above. 

See [**further details and help from here**](https://pythongis.org/part1/chapter-01/nb/04-using-jupyterlab.html). 

## Hints

If there are general questions arising from this exercise, we will add hints to the course website under [Exercise 3 description](https://spatial-analytics.readthedocs.io/en/latest/lessons/L3/exercise-3.html).

## Input data

In this exercise, we use data from an area in Northern Sweden representing **copper content of rock samples**. 

- The data is provided to you as a GeoJSON file which is located in `data/copper_data.geojson`. 
- In the data, you see three attributes `F1`, `F2` and `F3`. First two are horizontal coordinates of sample point in Swedish rectangular coordinate system. The third field `F3` is the relative copper content in soil as mg/kg (ppm). 
- The data has been obtained from the [Geological Survey of Sweden (SGU)](https://www.sgu.se/en/).


## Helper functions

The following functions are provided for you to make things easier. **You need to execute/run this cell before you can use them!**
Scroll down the notebook to find the Problems where you should use these functions for different tasks.
Hint: You might want to study these functions carefully, and especially the documentation provided for them (as docstrings) to understand how to use them.

There are altogether 6 functions below from which you will be using 5 in this exercise:

1. `create_hexagon_grid()`
2. `interpolate_idw()`
3. `get_semivariogram_model()`
4. `interpolate_ordinary_kriging()`
5. `calculate_RMSE()`

In [ ]:
def create_hexagon_grid(gdf, gridsize=60):
    """
    Generates a Hexagon-grid GeoDataFrame having the same extent as the input gdf. 
    
    Parameters
    ----------
    
    gdf: gpd.GeoDataFrame with points
        
        A GeoDataFrame containing points (note: other geometry types won't work)
        
    gridsize: int 
    
        Number of hexagons that should be created per side. Default 60.
    
    """
    import geopandas as gpd
    import matplotlib.pyplot as plt
    from shapely.geometry import Polygon
    import numpy as np

    # Get coordinates
    x = gdf.geometry.x.values
    y = gdf.geometry.y.values

    # Create the hexagons
    plt.axis('off')
    collection = plt.hexbin(x,y, gridsize=gridsize, 
                           alpha=0, linewidths=0)
    
    # Parse the hexagon geometries into Shapely Polygons
    hex_polys = collection.get_paths()[0].vertices
    hex_array = []
    for xs, ys in collection.get_offsets():
        hex_x = np.add(hex_polys[:,0],  xs)
        hex_y = np.add(hex_polys[:,1],  ys)
        hex_array.append(Polygon(np.vstack([hex_x, hex_y]).T))
        
    return gpd.GeoDataFrame({'geometry': hex_array}, crs=gdf.crs)


def interpolate_idw(known_locations_gdf, 
                    unknown_locations_gdf, 
                    value_column, 
                    target_column="predicted_idw",
                    n_neighbors=-1,
                    power=2
                   ):
    """
    A helper function to do IDW interpolation for locations defined in 'unknown_points_gdf'.
    The values in column `value_column' of the 'known_points_gdf' are used as the basis for doing the interpolation.
    
    Parameters
    ----------
    
    known_locations_gdf : gpd.GeoDataFrame
        
        GeoDataFrame containing the known values at given points that are used as the basis for the interpolation.
        The geometries can be either points or polygons (a centroid of Polygon will be extracted). 
        
    unknown_locations_gdf : gpd.GeoDataFrame
        
        GeoDataFrame containing the unknown locations. The geometries can be either points or 
        polygons (a centroid of Polygon will be extracted). The values for these locations will be predicted using the 
        Inverse Distance Weighting interpolation method. 
    
    value_column : str
        
        The value column in the `known_points_gdf' containing the known values (𝑤𝑖) used for prediction.
        
    target_column : str
        
        The column where the predicted value (𝑧̂) will be stored.
        
    n_neighbors : int
        
        Number of neighbors to use for weighting. By default, all neighbors are used in the prediction (i.e. value: -1)
        
    power : int
    
        The power (𝛽) that defines the distance decay function (by default: 2). Higher power value emphasize the influence of the 
        points nearest to the unknown point. A smaller power value gives more equal influence of also more distant points.
        
    """
    
    import numpy as np
    from pyinterpolate.idw import inverse_distance_weighting
    
    # Parse the pointset for the known locations (training dataset)
    known_points = get_pointset_for_interpolation(known_locations_gdf, value_column)

    # Create empty column for interpolated values
    unknown_locations_gdf[target_column] = None

    # Interpolate the values with the test dataset
    for row in unknown_locations_gdf.itertuples():
        index = row.Index
        point = np.array([row.geometry.centroid.x, row.geometry.centroid.y])
        # Do the prediction using IDW
        prediction = inverse_distance_weighting(known_points, 
                                                point, 
                                                number_of_neighbours=n_neighbors, 
                                                power=power)
        # Assign the value to the result
        unknown_locations_gdf.loc[index, target_column] = round(prediction, 4)
        
    return unknown_locations_gdf


def get_semivariogram_model(known_locations_gdf, 
                            value_column, 
                            search_radius, 
                            max_range,
                            weighting,
                            n_ranges,
                            nugget=0
                           ):
    
    """
    A helper function to find optimal semivariogram model for given points in 'gdf'.
    The values in column `value_column' of the 'gdf' are used as the empirical basis for finding the semivariogram model.
    
    Parameters
    ----------
    
    known_locations_gdf : gpd.GeoDataFrame
        
        GeoDataFrame containing the known values at given points that are used as the basis for the interpolation.
        The geometries can be either points (preferred) or polygons (a centroid of Polygon will be extracted). 
        
    value_column : str
        
        The value column in the `gdf' containing the known values.
        
    search_radius : int
        
        The column where the predicted value (𝑧̂) will be stored.
        
    max_range : int
        
        Number of neighbors to use for weighting. By default, all neighbors are used in the prediction (i.e. value: -1)
    
    n_ranges : int
    
        The model optimization algorithm divides the study extent into n number of ranges 
        and tests the theoretical model output against the experimental variogram for each range.
        
    nugget : float
        
        A nugget allows for the variogram to assume some measurement error in the original observations.
        Presence of a nugget means that any two observations sampled arbitrarily closely will not necessarily 
        have the same value. It can somewhat be thought as possible "measurement error" in the original observations. 

    Returns
    -------

    Theoretical semivariogram model
            
    """
    
    from pyinterpolate.variogram.empirical.experimental_variogram import calculate_semivariance
    from pyinterpolate import TheoreticalVariogram, build_theoretical_variogram, build_experimental_variogram
    
    # Extract x, y and z values from the gdf
    known_points = get_pointset_for_interpolation(known_locations_gdf, value_column)
    
    # Create experimental semivariogram (based on the known values at given locations)
    exp_semivar = build_experimental_variogram(input_array=known_points, step_size=search_radius, max_range=max_range)

    # Fit data into a theoretical model
    semivar = TheoreticalVariogram()
    fitted = semivar.autofit(experimental_variogram=exp_semivar, 
                         model_types='all', 
                         nugget=nugget,
                         number_of_ranges=n_ranges,
                         deviation_weighting=weighting,
                        )
    semivar.plot()
    print("[INFO] RMSE of the fitted model:", semivar.rmse)

    return semivar
    

def interpolate_ordinary_kriging(known_locations_gdf, 
                                 unknown_locations_gdf, 
                                 model, 
                                 value_col,
                                 target_col, 
                                 error_col, 
                                 neighbors_range=None,
                                 n_neighbors=8):
    """
    A helper function to do Ordinary Kriging interpolation for locations defined in 'gdf' 
    based on given semivariogram 'model'.
    
    Parameters
    ----------
    
    known_locations_gdf : gpd.GeoDataFrame
        
        GeoDataFrame containing the known values at given points that are used as the basis for the interpolation.
        The geometries can be either points or polygons (a centroid of Polygon will be extracted). 
        
    unknown_locations_gdf : gpd.GeoDataFrame
        
        GeoDataFrame containing the unknown locations. The geometries can be either points or 
        polygons (a centroid of Polygon will be extracted). The values for these locations will be predicted using the 
        Inverse Distance Weighting interpolation method. 
        
    model : pyinterpolate.TheoreticalVariogram
    
        Theoretical Variogram used for data interpolation. Use `get_semivariogram_model()` function to generate this model.
        
    value_col : str
    
        The value attribute in `known_locations_gdf` containing information measurements.
        
    target_col : str
        
        The column where the predicted values will be stored in the `unknown_locations_gdf`.
        
    error_col : str
        
        The column where the standard errors will be stored in the `unknown_locations_gdf`.
        
    neighbors_range : float
    
        None or float, the maximum distance for searching neighbors for a point. 
        If None is given, then the range is selected from the theoretical_model rang attribute.
        
    n_neighbors : int
        
        Number of neighbors to use for weighting. By default, 8 neighbors are used in the prediction (i.e. value: 8)
        
    """
    from pyinterpolate import kriging
    import numpy as np
    
    # Parse the points with known observations
    known_points = get_pointset_for_interpolation(known_locations_gdf, value_col)
    
    # Parse the coordinates of the unknown locations
    unknown_points = np.vstack([unknown_locations_gdf.geometry.centroid.x.values, 
                                unknown_locations_gdf.geometry.centroid.y.values]).T
    
    # The Kriging interpolation returns various things (as a tuple)
    # Here, we unpack the values into their own variables.
    # 'predicted' value for given location, standard 'error' for the estimate
    # 'weights' that were applied for given point 
    results = kriging(observations=known_points,
                      theoretical_model=model,
                      points=unknown_points,
                      how="ok",
                      neighbors_range=neighbors_range,
                      no_neighbors=n_neighbors,
                   ).T

    predicted, error, estimated_mean, weights = results[0], results[1], results[2], results[3]

    # Store the results
    unknown_locations_gdf[target_col] = np.round(predicted, 2)
    unknown_locations_gdf[error_col] = error
    
    return unknown_locations_gdf



def calculate_RMSE(gdf, validation_col="F3", prediction_col="predicted_value"):
    """
    A helper function to calculate the Root Mean Square Error (RMSE) for predicted values. 
    The predictions are compared against the known values (validation). The smaller the value, the better.
    
    Parameters
    ----------
    
    gdf: gpd.GeoDataFrame
        
        GeoDataFrame that should contain both the observed and predicted values.
        
    validation_col : str
        
        Column name for the observed values (i.e. the values that we know beforehand)
        
    prediction_col : str
        
        Column name for the predicted values (i.e. the values that were estimated using interpolation)
    
    """
    
    from sklearn.metrics import mean_squared_error
    return mean_squared_error(gdf[validation_col].values, gdf[prediction_col].values, squared=False)


def get_pointset_for_interpolation(gdf, value_column):
    """
    A helper function to extract a pointset (numpy arrays) for interpolation. 
    
    Parameters
    ----------
    
    gdf: gpd.GeoDataFrame 
        
        Input GeoDataFrame from which the pointset for interpolation are parsed. The x and y coordinates
        are parsed from the geometry, and the z values are parsed from the selected column.
        In case the input geometries are polygons, a centroid of the given geometry is returned.
        
    value_column : str
        
        The name of the column that contains the values (z) for the attribute of interest 
        (e.g. the amount of copper observed at given location). 
    """
    import numpy as np
    # Extract x, y coordinates and the attribute (z) values from the gdf 
    x = gdf.geometry.centroid.x.values
    y = gdf.geometry.centroid.y.values
    z = gdf[value_column].values
    return np.array([x, y, z]).T

## Problem 1 - Preparations

### Task 1.1 - Divide the data into training and test datasets (4 points)

**1. Read the data** file `data/copper_data.geojson` with geopandas into a variable called `data`(*the result will be a GeoDataFrame*).

**2. Generate an interactive map** using the [`.explore()` function](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html) of geopandas, in which you should visualize the copper content in the column `"F3"`. You can use the classification scheme `quantiles` to better differentiate the patterns in the data (pass `scheme="quantiles"` parameter into the `.explore()` function). See an example map below. 

**3. Divide the data into training and test datasets** by randomly picking 75 % of the data for training and 25 % for test:
  - Store the training data into variable `train` and test data into variable `test`
  - **Hint 1**: For random sampling the 75 % of the data into variable `train`, you can use the `.sample()` method of pandas (see [docs for help](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html))
  - **Hint 2**: For selecting the remaining 25 % of the data, you should pick the rows the `data` which were **not** selected into the 75 % random sample. For doing this, you can take advantage of the index values in the `data` and `train`, and conduct an inverse selection using tilde (`~`) and `isin()` method. See [this StackOverflow answer](https://stackoverflow.com/a/44318806) for help.

**4. Genererate an interactive map** which shows the training points with red color and the test points with blue color. See an example map below.
  - **Hint:** Adding multiple layers with `.explore()` is possible as shown in the [last example on this page](https://geopandas.org/en/stable/docs/user_guide/interactive_mapping.html#Interactive-mapping). The examples on the page also shows how you can adjust different visualization parameters, such as color of the points.


![Copper content](img/copper_content.png)
*Example 1: The copper content visualized using quantiles classification (step 2 above).*

![Training and test points](img/training_and_test_points.png)

*Example 2: Randomly sampled points for training (75 %) and testing (25 %). Notice that the pattern you will see is likely different because they are picked randomly. (step 4 above)*

In [14]:
import geopandas as gpd

"""Read the data file data/copper_data.geojson with geopandas into a variable 
called data(the result will be a GeoDataFrame)."""

path = "data/copper_data.geojson"
data = gpd.read_file(path)
data = gpd.GeoDataFrame(data)
print(data)


"""Generate an interactive map using the .explore() function of geopandas, 
in which you should visualize the copper content in the column "F3". 
You can use the classification scheme quantiles to better differentiate 
the patterns in the data (pass scheme="quantiles" parameter into the 
.explore() function). See an example map below. """

#data.explore("F3", scheme="quantiles")


"""Divide the data into training and test datasets by randomly picking
75 % of the data for training and 25 % for test.
Store the training data into variable train and test data into variable test"""

train = data.sample(frac=0.75, random_state=1)
print(train)
test = data[~data.index.isin(train.index)]
print(test)


"""Genererate an interactive map which shows the training points with red color
and the test points with blue color. """

map = train.explore(color="red", legend=True)
test.explore(color="blue", legend=True, m=map)
map



             F1         F2    F3                         geometry
0     7204378.0  1646028.0   8.0  POINT (1646028.000 7204378.000)
1     7191962.0  1646295.0   9.0  POINT (1646295.000 7191962.000)
2     7196825.0  1646308.0  15.0  POINT (1646308.000 7196825.000)
3     7183878.0  1646582.0   7.0  POINT (1646582.000 7183878.000)
4     7165608.0  1646905.0  14.0  POINT (1646905.000 7165608.000)
...         ...        ...   ...                              ...
1040  7153486.0  1645425.0  28.0  POINT (1645425.000 7153486.000)
1041  7169636.0  1645600.0  18.0  POINT (1645600.000 7169636.000)
1042  7161772.0  1645647.0  14.0  POINT (1645647.000 7161772.000)
1043  7157299.0  1645682.0  10.0  POINT (1645682.000 7157299.000)
1044  7176381.0  1645717.0   8.0  POINT (1645717.000 7176381.000)

[1045 rows x 4 columns]
             F1         F2    F3                         geometry
949   7147270.0  1634339.0  17.0  POINT (1634339.000 7147270.000)
94    7149358.0  1656493.0  18.0  POINT (1656493.00

### Task 1.2 - Generate a hexagon grid (2 points)

The hexagon grid will be used for producing continuous interpolation surface in the problems 2 and 3.

**1. Generate a hexagon grid** (as below) by using the `create_hexagon_grid()` function provided for you in the [Helper functions](#Helper-functions). **Store the result** into variable `grid`. You can use the default gridsize (i.e. 60). 
  - Hint: Read the function docstring in [Helper functions](#Helper-functions) to understand how it works.
**2. Generate an interactive map** showing the grid using `.explore()` that should look something like below:

![Hexagon grid](img/hexagon_grid.png)

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

## Problem 2 - Interpolation with IWD

In this problem, the objective is to conduct interpolation using Inverse Distance Weighting algorithm, and test how changing the `power` influences the results and accuracy of the prediction. All the interpolations that we do in this exercise are all based on the [`pyinterpolate` -library](https://pyinterpolate.readthedocs.io/en/latest/).

### Task 2.1 - Find out the best performing $\beta$ in IDW (3 points)

**1. Use the `interpolate_idw()` function to conduct IDW interpolation** which is provided for you in [Helper functions](#Helper-functions) (read the documentation/docstring of the function for further details):

  - You should use the `train` dataset as the known data points
  - Use the `test` dataset as the points which are unknown, i.e. the predictions will be done for those locations (and saved into this table)
    - Notice that in reality, we do know the real copper values for the points in `test` but here we "pretend" that we don't know them, because in this way we can assess the accuracy of our prediction.
  - The column `"F3"` in the `train` dataset contains the values (copper content) that we want to use in prediction
  - Use **a power of 1** in the interpolation (i.e. $\beta$)
  - Specify that you want to store the result in a **column called `"idw_power_1"`**
  - Store the result from this function into a variable `test` (i.e. the `test` dataset that was passed as one of the argument will be updated)
    
**2. Calculate the Root Mean Square Error (RMSE) for the prediction** using the `calculate_RMSE()` -function that is provided for you in [Helper functions](#Helper-functions) (read the documentation/docstring of the function for further details how to use it):

  - Pass the result from step 1 (i.e. should be the variable `test`)
  - Use the `"F3"` column for the validation
  - Use the column `"idw_power_1"` as the `prediction_col`
  - Store the result into a new variable called `idw_p1`
  - Print out the RMSE for the interpolation showing how well the predicted values compare against the known values. The lower the RMSE, the better the prediction.
  
**3. Repeat the steps 1 and 2** and test how **changing the power** ($\beta$) influences the accuracy of the result. 

  - Repeat the analysis with **power values: 2,3 and 4**. Store the results into variables `idw_p2`, `idw_p3` and `idw_p4` accordingly. 

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

### Questions

*Correct answers gives you 0.5 points allocated to problem 2*.

 - Question 2.1: Which $\beta$ value produced the **best** result in your analysis above? 
 - Question 2.2: Which $\beta$ value produced the **worst** result? Why do you think it performs poorly?

### Answers

Answer to the questions by adding text after the `Answer` bullet points below:

(hint: double-click this cell to activate editing)

- Answer for Q2.1:
- Answer for Q2.2:

### Task 2.2 - Create a map with IDW predictions (2 points)

Use the power that had the best performance in the previous step, and create a map using the hexagon grid created in Task 1.2.

**1. Use the `interpolate_idw()` function** and:
  
  - use the `train` dataset as the known locations
  - use the hexagon `grid` as the unknown locations to which we will predict values using IDW
  - use the $\beta$ that performed best
  - use again the `"F3"` as the column for known values
  - specify that the results should be stored into column `idw_power_X`, where you should replace the `X` with the power that you choose to use
  - store the result from the function into variable `grid` (i.e. we will update the hexagon grid created earlier)
  
**2. Create a map with the hexagon grid** where the color of the cell should be the interpolated value from step 1. Use a colormap `"Reds"` that you can define with parameter `cmap`. If you want, you can add the `test` points on top of this hexagon grid which allows you to easily evaluate visually how well the predictions have worked at different locations. As a result, you should get something like the map below.

  - **Hint:** Adding multiple layers with `.explore()` is possible as shown in the [last example on this page](https://geopandas.org/en/stable/docs/user_guide/interactive_mapping.html#Interactive-mapping). The examples on the page also shows how you can adjust different visualization parameters, such as color of the points.
  
![IDW interpolation](img/idw_interpolation.png)

*Example map how the result from IDW interpolation should look like (approximately).*

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

## Problem 3 - Interpolation with Kriging

In this problem, we will continue working with the same data and use **Ordinary Kriging** to predict the values (copper content) for unknown locations.
To do this, we will:

1. Check whether the data is normally distributed. If not:
2. Transform the data by taking a log of the values, which helps to "normalize" the data
3. Find the optimal semivariogram model for the data, and test how changing the parameters influence the model fit
4. Conduct Ordinary Kriging for the `test` dataset with the selected model and validate the result by calculating the RMSE 
5. Visualize the interpolation result as well as the standard errors that shows where the prediction works well and where not

### Task 3.1 - Testing the assumptions (2 points)

**1. Is the data normally distributed?**:

  - Plot a histogram of the `"F3"` values using the `.plot.hist()` function of pandas ([see docs](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.hist.html) for help). Use 30 bins which helps showing the differences at different value classes. 
  - To make a better visual assessment of the normality, also make a [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot) that helps to estimate whether your data points follow normal distribution. For this, you can use the [`qqplot()` -function](https://www.statsmodels.org/stable/generated/statsmodels.graphics.gofplots.ProbPlot.qqplot.html) from the `statsmodels` library. See [this StackOverflow answer](https://stackoverflow.com/a/50677734) for reference.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

**2. Log transform the data** (if it does not follow normal distribution):

In case your data was not normally distributed, it needs to be transformed in such a way that it follows normal distribution. There are a few different approaches for doing this, and one of the most common approaches is to log-transform your data which helps to make the data as "normal" as possible. (*However, note that log-transform is not always recommended e.g. [with count data having zeros](https://besjournals.onlinelibrary.wiley.com/doi/10.1111/j.2041-210X.2010.00021.x)*).

  - You can log-transform you data by using the  [`np.log()` -function](https://numpy.org/doc/stable/reference/generated/numpy.log.html) and applying it for the `"F3"` column. 
  - Store the data into a new column called `"log_F3"`
  - Test with Q-Q plot and histogram whether the data now follows better the normal distribution.

In [ ]:
# Add your code here IF NEEDED

# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

### Questions

*Sufficient description gives you 0.5 points allocated to problem 3*.

- Question 3.1: Based on the plots you made above, do you think the data follows normal distribution? Justify your answer.

### Answers

Answer to the questions by adding text after the `Answer` bullet point.

(hint: double-click this cell to activate editing)

- Answer for Q3.1: 

## Task 3.2 - Find optimal semivariogram model (2 points)

As a background information for this task, you can check the [documentation](https://pyinterpolate.readthedocs.io/en/stable/usage/tutorials/Semivariogram%20Estimation%20%28Basic%29.html#4)-Set-automatically-semivariogram-model) to understand how the optimal model can be found using the `pyinterpolate` library. Also it is **highly recommended** that you check the [slides for geostatistics](https://spatial-analytics.readthedocs.io/en/latest/lessons/L2/geostatistics-kriging.html) and [watch the lesson video](https://spatial-analytics.readthedocs.io/en/latest/lessons/L2/overview.html#lesson-videos), in case you want to revise what was said about the things relating to the questions below.

**1. Investigate your data** using the interactive maps done in earlier tasks, and update the variables (also just test how changing the values influences the result!):
    
  - Think what would be a good **search radius** for the data? The search radius defines the distance between lags within each points are included in the calculations. 
    - **Modify the value for the `search_radius` variable below** (should be in meters). 
    - **Question 3.2: Describe why/how did you end up selecting the given search radius value?** (answer below to the "Answers" cell)
  
  - Investigate the interactive maps, and think what would be the maximum possible distance between two points in the dataset? 
    - **Update the `max_range` variable** below based on your findings (should be in meters).
    - Hint: take advantage of the scalebar on the bottom left corner of the interactive maps and look the distances between points. The map and scalebar updates when you zoom in/out. 
  
  - Think whether the influence of the nearest points for given location should be **weighted in some way**? 
  
    - **Update the `weighting` variable** below (possible values are: equal, closest, furthest) and test different values for it. 
    - **Question 3.3**: Which of them works the best? Describe why you ended up using specific `weighting` parameter in your analysis? (answer below to the "Answers" cell)
    
  - Test whether allowing nugget, i.e. some error in the original measurements influences the fit of the Theoretical Model:
  
    - **Update the `nugget` variable** below (test with values: 0, 0.02, 0.05) and test how it influences the fit of your model.
    - **Question 3.4**: Which of the nugget values produces the "best" result? Why? (answer below to the "Answers" cell)

**2. Get the optimal semivariogram model** based on your parameters defined in step 1:

   - **Use the `get_semivariogram_model()`** provided for you in the [Helper functions](#Helper-functions) (read the documentation/docstring of the function for further details how to use it).
   - Use the `train` dataset again as the input gdf
   - Use the `"F3"` or `"log_F3"` attribute as value column, depending on whether you transformed the data or not
   - Pass the `search_radius`, `max_range`, `weighting`, `number_of_ranges` and `nugget` variables for the function (check the function docstring to know how the parameters are called)
   - Save the resulting model that is returned by the `get_semivariogram_model()` into a variable `model`
   
**This model is used for determining the weights for the interpolation**, and in practice used for doing the interpolation (see next step).    
As a result of this, you should end up having something like following (it can look different due to different parameter values, and it's okay):
![Semivariogram model](img/semivariogram_model.png) 

In [ ]:
# Search radius (in meters in this case because our points are in metric system)
search_radius = 0

# The maximum distance between points
max_range = 0

# Weighting
# Possible values: "equal", "closest", "furthest" 
weighting = "TEST_DIFFERENT_WEIGHTING"

# Algorithm divides the study extent into n number of ranges and tests 
# the theoretical model output against the experimental variogram for each range
# Note: This can be as it is.
number_of_ranges = 20

# Nugget - I.e. how much do we allow "measurement error" in the original observations
# Test different values: 0, 0.02, 0.05 
nugget = 0.05

# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

### Questions

*Sufficient description gives you 1 point allocated to problem 3.2*.

- **Question 3.2**: Describe why/how did you end up selecting the given search radius value?
- **Question 3.3**: Which of the `weighting` works the best? Describe why you ended up using specific `weighting` parameter in your analysis?
- **Question 3.4**: Which of the `nugget` values produces the "best" result? Why? 


### Answers

Answer to the questions by adding text after the `Answer` bullet point.

(hint: double-click this cell to activate editing)

- Answer for Q3.2: 
- Answer for Q3.3: 
- Answer for Q3.4: 

## Task 3.3 - Interpolate with Ordinary Kriging (2 points)

After you have defined and created a semivariogram model in the previous step, we can do the interpolation.

**1. Use the `interpolate_ordinary_kriging()` -function** provided for you in the [Helper functions](#Helper-functions) (read the documentation/docstring of the function for further details how to use it). 

  - Use the `train` dataset as the `known_locations_gdf` for the function (i.e. the data with measured observations)
  - Use the `test` dataset as the `unknown_locations_gdf` for the function (i.e. the predictions will be done for these points and stored to this GeoDataFrame)
  - Use the semivariogram `model` created in the previous step representing the Theoretical Semivariogram used for data interpolation.
  - Use the `F3` (or logged version of it) as the `value_col`, which contain information about the values of the measured observations.
  - Give an intuitive name for the paramater `target_col` where the predicted values will be stored, such as `"predicted_krige"` or `"predicted_krige_log"`
  - Give an intuitive name for the parameter `error_col` where the standard error of the predictions will be stored, such as `"std_error"` or `"std_error_log"`
  - With `neighbors_range` parameter, you can control the search radius for the known observations which are used to make the predictions.
    - Alternatively with `n_neighbors` parameter, you can control how many closest neighbors of the known points are used for making the predicions.
  
  
**2.** In case you have transformed the data, **back-transform the interpolation predictions and standard errors** by calculating the exponential of all predicted and standard error values:

  - Store the back-transformed values into `test` dataset with intuitive column names, such as `predicted_krige` and `std_error`
  - You can calculate the exponential by using the [`np.exp()` -function](https://numpy.org/doc/stable/reference/generated/numpy.exp.html)
  - After this step, your predicted values are on the same scale as the original values in `"F3"` column.
  
**3. What is the RMSE for the Kriging predictions?**

  - Calculate the Root Mean Square Error (RMSE) for the prediction** using the `calculate_RMSE()` -function that is provided for you in [Helper functions](#Helper-functions) (read the documentation/docstring of the function for further details how to use it):

  - Pass the `test` GeoDataFrame as the gdf
  - Use the `"F3"` column for the validation
  - Use the `"predicted_krige"` column as the `prediction_col` (i.e. the column having the values on the same scale as `"F3"`)
  - Store the result into a new variable called `krige_rmse`
  - Print out the RMSE for the interpolation showing how well the predicted values compare against the known values. The lower the RMSE, the better the prediction. 

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

### Questions

*Sufficient description gives you 0.5 points allocated to problem 3.3*.

- **Question 3.5**: Did the Kriging interpolation perform better than IDW? Justify your answer with a short description.

### Answers

Answer to the questions by adding text after the `Answer` bullet point.

(hint: double-click this cell to activate editing)

- Answer for Q3.5: 

### Task 3.4 - Create a map with Ordinary Kriging predictions (2 points)

**1. Use the `interpolate_ordinary_kriging()` function** and:
  
  - use the hexagon `grid` as the unknown locations to which we will predict values using IDW
  - Use the semivariogram `model` created in the previous step representing the Theoretical Semivariogram used for data interpolation.
  - Give an intuitive name for the column where the predicted values will be stored, such as `"predicted_krige"` or `"predicted_krige_log"`
  - Give an intuitive name for the column where the standard error of the predictions will be stored, such as `"std_error"` or `"std_error_log"`
  - Store the result from the function into variable `grid` (i.e. we will update the hexagon grid created earlier)
  - If needed, back-transform the values again in a similar manner as in Task 3.3.
  
**2. Create a map with the predicted values using the hexagon grid**, where the color of the cell should be the predicted values from step 1. Use a colormap `"Reds"` that you can define with parameter `cmap`. If you want, you can again add the `test` points on top of this hexagon grid which allows you to easily evaluate visually how well the predictions have worked at different locations. As a result, you should get something like the map below.

  - **Hint:** Adding multiple layers with `.explore()` is possible as shown in the [last example on this page](https://geopandas.org/en/stable/docs/user_guide/interactive_mapping.html#Interactive-mapping). The examples on the page also shows how you can adjust different visualization parameters, such as color of the points.

**3. Create a map with the standard error values using the hexagon grid**, where the color of the cell should be the standard error values. Use a colormap `"Blues"` that you can define with parameter `cmap`. As a result, you should get something like the map below.

\*_Further info related to Question 1: Clusters or trends in the standard errors would indicate that it would be good to try to remove the trends and consider anisotropy in the interpolation process. We won't be considering these things in this exercise, but if you are interested to learn more and take into account anisotropy there is another Python package called [PyKrige](https://geostat-framework.readthedocs.io/projects/pykrige/en/stable/index.html) that [can handle anisotropy](https://geostat-framework.readthedocs.io/projects/pykrige/en/stable/generated/pykrige.ok.OrdinaryKriging.html). The library also provides [Universal Kriging](https://geostat-framework.readthedocs.io/projects/pykrige/en/stable/generated/pykrige.uk.UniversalKriging.html) method allowing to use drift terms._ 

![Ordinary Kriging predictions example](img/ordinary_kriging_predictions.png)

___Example 1__ of the resulting map for the Ordinary Kriging predicted values._


![Ordinary Kriging std errors example](img/ordinary_kriging_std_errors.png)

___Example 2__ of the resulting map for the Ordinary Kriging standard error values._


In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

### Questions

*Sufficient description gives you 1 point allocated to problem 3.4*.

- **Question 3.6**: Do you think the result based on Ordinary Kriging differs significantly from the IDW interpolation? Give a short explanation.
- **Question 3.7**: Do you see any evident patterns in the standard error map? Give a short explanation. 
- **Question 3.8** What is the purpose of standard error map? Why is it useful?

### Answers

Answer to the questions by adding text after the `Answer` bullet point.

(hint: double-click this cell to activate editing)

- Answer for Q3.6: 
- Answer for Q3.7: 
- Answer for Q3.8: 

## Problem 4 - How long did it take? Optional feedback (1 point)

To help developing the exercises, and understanding the time that it took for you to finish the Exercise, please provide an estimate of how many hours you spent for doing this exercise?

 - I spent approximately this many hours: **X hours**
 
In addition, if you would like to give any feedback about the exercise (optional), please provide it below:

 - My feedback: